add any imports below

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import sys
import csv
from collections import Counter
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import words, stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import torch
from transformers import BertTokenizer, BertModel
import re

In [ ]:
# increasing field size limit so csv file can be read
csv.field_size_limit(sys.maxsize)

131072

Starting by downloading and importing the data from: https://www.kaggle.com/c/fake-news/data?select=train.csv

In [ ]:
# making pandas dataframe from csv file
train_df = pd.read_csv('train.csv', engine='python')
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


this dataframe has over 20,000 articles in it. to make it easier to work with for this project, we are using a subsample of 5000 documents, 2500 that are reliable and 2500 that aren't reliable

In [ ]:
# dropping rows with "na" in text since this is the feature we are using
# dropping rows that only contain blank spaces
train_df = train_df[train_df['text'].notna()]
#train_df = train_df[train_df['text'] != ' ']
train_df = train_df[train_df['text'].apply(lambda x: bool(re.search('[a-zA-Z0-9]', x)))]
count = 0
for i in train_df['text'].tolist():
  if i == []:
    print(count)
  if (len(i) < 5):
    print(i, count)
  count+=1

f 779
liar 5669
No. 7091
Cool 13230


a label of 1 is unreliable, 0 is reliable

let's do some brief analysis of the data

In [ ]:
print("The training dataset contains", len(train_df['title']), "news articles.")
print("There are", len(train_df[train_df['label']==1]), "unreliable news sources.")
print("There are", len(train_df[train_df['label']==0]), "reliable news sources.")

The training dataset contains 20671 news articles.
There are 10284 unreliable news sources.
There are 10387 reliable news sources.


using BERT to create word vectors

Load pre-trained BERT model and tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Tokenize text and convert to input features for each article

In [ ]:
inputs_list = []
count = 0
max_length=512
#print((train_df['text'].tolist()[893]) == "  ")

for article in train_df['text'].tolist():
    tokens = tokenizer.tokenize(article)
    inputs = tokenizer.encode(tokens, return_tensors='pt', max_length = max_length, padding='max_length')
    inputs_list.append(inputs)

    count = count + 1
    if count % 500 == 0:
      print(count)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500


 Generate embeddings using BERT for each article

In [ ]:
#!rm -r embeddings

In [ ]:

import time
import torch

outputs_list = []
count = 0
model.cuda()
s = time.time()
with torch.no_grad():
    for i in range(0, len(inputs_list)):
        input = inputs_list[i]
        output = model(input[:, :512].cuda())
        output = output.last_hidden_state
        count = count + 1
        # Save outputs_list in batches
        torch.save(output, f"embeddings/outputs_list_{count}.pth")
        # Free up GPU CUDA memory
        torch.cuda.empty_cache()
        if count % 1000 == 0:
            print(f"Embeddings done: {count}")
            print(time.time() - s)
    print(f"Total embeddings: {count}")

print(time.time() - s)


Embeddings done: 1000
35.769510984420776
Embeddings done: 2000
74.67981457710266
Embeddings done: 3000
115.91610074043274
Embeddings done: 4000
156.5756447315216
Embeddings done: 5000
197.42531490325928
Embeddings done: 6000
238.39382934570312
Embeddings done: 7000
279.3616840839386
Embeddings done: 8000
320.19804072380066
Embeddings done: 9000
361.0397629737854
Embeddings done: 10000
401.8389265537262
Embeddings done: 11000
442.83367252349854
Embeddings done: 12000
483.65119886398315
Embeddings done: 13000
524.6044158935547
Embeddings done: 14000
565.5098280906677
Embeddings done: 15000
606.4263460636139
Embeddings done: 16000
647.3428728580475
Embeddings done: 17000
689.0246303081512
Embeddings done: 18000
729.9058158397675
Embeddings done: 19000
770.7414910793304
Embeddings done: 20000
811.5646526813507
Total embeddings: 20671
838.9626228809357


TESTING CODE

In [ ]:

len(train_labels)

14500

torch.load(....pth) loads in the word embeddings in a list of length 2000 where each tensor represents an article! Shape is [1, 512, 768]

starting model implementatiion here
https://www.analyticsvidhya.com/blog/2020/01/first-text-classification-in-pytorch/

define hyperparameters for model

In [ ]:
batch_size = 16
embedding_dim = 768 # can't change this!!!
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
train_split_ct = 14500
val_split_ct = 17600
test_split_ct = 20670

train_labels = train_df['label'].tolist()[:train_split_ct]
val_labels = train_df['label'].tolist()[train_split_ct:val_split_ct]
test_labels = train_df['label'].tolist()[val_split_ct:test_split_ct]

In [ ]:
import torch
import builtins
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
class LSTM(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional):
    super(LSTM, self).__init__()

    self.lstm = nn.LSTM(embedding_dim,
                        hidden_dim,
                        num_layers=n_layers,
                        bidirectional=bidirectional,
                        dropout=.2,
                        batch_first=True)

    self.fc = nn.Linear(hidden_dim * 2, output_dim)

    self.sigmoid = nn.Sigmoid()

  def forward(self, input):
    input = torch.squeeze(input)
    output, (hidden, cell) = self.lstm(input)
    # hidden = [batch size, num layers * num directions, hid dim]
    # cell = [batch size, num layers * num directions, hid dim]

    hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)

    dense_outputs = self.fc(hidden)

    outputs = self.sigmoid(dense_outputs)

    return outputs



Hyperparameters for training

In [ ]:
learning_rate = .0001
num_epochs = 20
print_every = 10

In [ ]:
model = LSTM(embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, bidirectional=True)
model.cuda()

LSTM(
  (lstm): LSTM(768, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

**Dataset and Dataloader**

In [ ]:
print(torch.tensor(train_df['label'].tolist()))
print(len(train_df['label']))

tensor([1, 0, 1,  ..., 0, 1, 1])
20671


In [ ]:
class BERT_dataset(Dataset):
    def __init__(self, dir, mode='train'):
        self.dir = dir
        if mode == 'train':
          self.labelList = torch.tensor(train_labels)
        elif mode == 'val':
          self.labelList = torch.tensor(val_labels)
        else:
          self.labelList = torch.tensor(test_labels)
        self.mode = mode

    def __len__(self):
        ## TODO: Return number of images in the dataset
        if self.mode == 'train':
          return len(train_labels)
        elif self.mode == 'val':
          return len(val_labels)
        else:
          return len(test_labels)

    def __getitem__(self, idx):
        ## return image, and, if not test mode, the label at the given index
        ## valid formats include:
        ## {'image': image, 'label': label}
        ## image, label
        ## where image and label have both been converted to torch.Tensors
        if self.mode == 'train':
          self.data = torch.load(f"{self.dir}{idx+1}.pth")
        elif self.mode == 'val':
          self.data = torch.load(f"{self.dir}{idx+train_split_ct+1}.pth")
        else:
          self.data = torch.load(f"{self.dir}{idx+val_split_ct+1}.pth")
        return self.data,self.labelList[idx]

In [ ]:
trainDataset = BERT_dataset('embeddings/outputs_list_')
trainDataloader = DataLoader(trainDataset,batch_size=batch_size,shuffle=True)

valDataset = BERT_dataset('embeddings/outputs_list_', mode = 'val')
valDataloader = DataLoader(valDataset,batch_size=batch_size,shuffle=True)

testDataset = BERT_dataset('embeddings/outputs_list_', mode = 'test')
testDataloader = DataLoader(testDataset,batch_size=batch_size,shuffle=False)

In [ ]:
def trainLoader(model, optimizer, criterion, train_loader):
  epoch_loss = 0
  num_correct = 0.0
  num_total = 0.0
  for batch_inputs, batch_labels in train_loader:
    # Reset the gradients
    optimizer.zero_grad()

    # Forward pass
    batch_outputs = model(batch_inputs)

    # Compute the loss
    batch_loss = criterion(torch.squeeze(batch_outputs), batch_labels.float().to("cuda"))

    # Backward pass
    batch_loss.backward()

    # Update the parameters
    optimizer.step()

    # Update the epoch loss
    epoch_loss += batch_loss.item()

    # Compute the number of correct predictions
    batch_predictions = (torch.squeeze(batch_outputs) >= 0.5).long()
    num_correct += (batch_predictions == batch_labels.float().to("cuda")).sum().item()
    num_total += len(batch_labels.float().to("cuda"))

  return epoch_loss, num_correct, num_total

In [ ]:
def train(model, optimizer, criterion, num_epochs=num_epochs, print_every=print_every):
  for epoch in range(num_epochs):
    epoch_loss = 0.0
    num_correct = 0.0
    num_total = 0.0
    length = 0

    model.train()
    length += len(trainDataloader)
    #!rm $name
    batch_epoch_loss, batch_num_correct, batch_num_total = trainLoader(model, optimizer, criterion, trainDataloader)
    epoch_loss += batch_epoch_loss
    num_correct += batch_num_correct
    num_total += batch_num_total


    # Compute the average epoch loss
    avg_epoch_loss = epoch_loss / length

    # Compute training accuracy
    train_acc = num_correct / num_total

    # Print the epoch loss
    if epoch % print_every == 0:
      print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_epoch_loss:.4f}, Training Accuracy: {train_acc:.4f}")

    #Calculate and print Val loss/accuracy
    evaluate(model, criterion, valDataloader)

In [ ]:
def evaluate(model, criterion, val_loader):
  # Put your model in evaluation mode
  model.eval()

  # Turn off gradient computation for evaluation
  with torch.no_grad():
      val_loss = 0.0
      num_correct = 0
      num_total = 0

      for batch_inputs, batch_labels in val_loader:
          # Forward pass
          batch_outputs = model(batch_inputs)

          # Compute the loss
          batch_loss = criterion(torch.squeeze(batch_outputs), batch_labels.float().to("cuda"))

          # Update the validation loss
          val_loss += batch_loss.item()

          # Compute the number of correct predictions
          batch_predictions = (torch.squeeze(batch_outputs) >= 0.5).long()
          num_correct += (batch_predictions == batch_labels.float().to("cuda")).sum().item()
          num_total += len(batch_labels.float().to("cuda"))

      # Compute the average validation loss
      avg_val_loss = val_loss / len(val_loader)

      # Compute the validation accuracy
      val_acc = num_correct / num_total

      # Print the validation loss and accuracy
      print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

In [ ]:
train(model, optimizer, criterion, num_epochs=num_epochs, print_every=1)

Epoch 1/20, Training Loss: 0.3594, Training Accuracy: 0.8512
Validation Loss: 0.2061, Validation Accuracy: 0.9284
Epoch 2/20, Training Loss: 0.1739, Training Accuracy: 0.9397
Validation Loss: 0.1532, Validation Accuracy: 0.9455
Epoch 3/20, Training Loss: 0.1311, Training Accuracy: 0.9559
Validation Loss: 0.1327, Validation Accuracy: 0.9497
Epoch 4/20, Training Loss: 0.1029, Training Accuracy: 0.9659
Validation Loss: 0.1277, Validation Accuracy: 0.9565
Epoch 5/20, Training Loss: 0.0829, Training Accuracy: 0.9732
Validation Loss: 0.1209, Validation Accuracy: 0.9581


KeyboardInterrupt: ignored